# Multiagent Pattern - Multiagent Collaboration

<img src="../img/multi_agent_pattern.png" alt="Alt text" width="500"/>

We follow the concept of crewai with our implementation. We create a crew which is a structured group of AI agents that collaborate to complete complex tasks together. 

Each agent has a distinct role. We declare dependencies between agents. This way the output of one agent, can be the input for another agent.


In [1]:
#%pip install -r requirements.txt
%pip install --upgrade pip
%pip install jupyter ipykernel
%%python3 -m ipykernel install --user --name=python310 --display-name "Python 3.10"


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


UsageError: Line magic function `%%python3` not found.


In [2]:
import sys
print("Python version:", sys.version)

Python version: 3.10.14 (main, Mar 19 2024, 21:46:16) [Clang 15.0.0 (clang-1500.1.0.2.5)]


In [3]:
%pip install openai python-dotenv graphviz colorama

Note: you may need to restart the kernel to use updated packages.


In [4]:
import openai
import dotenv
import graphviz
import colorama
print("✅ All dependencies installed successfully!")

✅ All dependencies installed successfully!


In [5]:
import sys
import os

project_root = os.getcwd()
src_path = os.path.join(project_root, "src")
if src_path not in sys.path:
    sys.path.append(src_path)

print("Updated sys.path:", sys.path)

Updated sys.path: ['/usr/local/Cellar/python@3.10/3.10.14/Frameworks/Python.framework/Versions/3.10/lib/python310.zip', '/usr/local/Cellar/python@3.10/3.10.14/Frameworks/Python.framework/Versions/3.10/lib/python3.10', '/usr/local/Cellar/python@3.10/3.10.14/Frameworks/Python.framework/Versions/3.10/lib/python3.10/lib-dynload', '', '/Users/glin/Documents/GitHub/M-LLAP/venv2/lib/python3.10/site-packages', '/Users/glin/Documents/GitHub/M-LLAP/design_patterns/src']


## The Agent Class

First of all, we need an **Agent Class**. This class implements an Agent, and internally it implements the ReAct technique.

In [6]:
import sys
import os
os.environ["OPENAI_API_KEY"]=""
print("API Key set:", os.getenv("OPENAI_API_KEY") is not None)


API Key set: True


In [7]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.multi_agent.agent import Agent
print("✅ Agent imported successfully!")

✅ Agent imported successfully!


Let's create some example agent, to see how it works.

You can also associate tools with the agent. Let's create a tool for writing some string into a CSV.

In [8]:
from src.tool_agent.tool import tool

In [9]:
@tool
def write_str_to_txt(string_data: str, txt_filename: str):
    """
    Writes a string to a txt file.

    This function takes a string and writes it to a text file. If the file already exists, 
    it will be overwritten with the new data.

    Args:
        string_data (str): The string containing the data to be written to the file.
        txt_filename (str): The name of the text file to which the data should be written.
    """
    # Write the string data to the text file
    with open(txt_filename, mode='w', encoding='utf-8') as file:
        file.write(string_data)

    print(f"Data successfully written to {txt_filename}")

In [16]:
agent_tool_example = Agent(
    name="Writer Agent",
    backstory="You are a language model specialised in writing text into .txt files",
    task_description="Write the string 'This is a Tool Agent' into './tool_agent_example.txt'",
    task_expected_output="A .txt file containing the given string",
    tools=write_str_to_txt,
)

In [17]:
agent_tool_example.run()


Thought: I need to write the string 'This is a Tool Agent' into a text file named 'tool_agent_example.txt'.

Using Tool: write_str_to_txt

Tool call dict: 
{'name': 'write_str_to_txt', 'arguments': {'string_data': 'This is a Tool Agent', 'txt_filename': 'tool_agent_example.txt'}, 'id': 0}
Data successfully written to tool_agent_example.txt

Tool result: 
None

Observations: {0: None}


"The string 'This is a Tool Agent' has been successfully written to 'tool_agent_example.txt'."

Let's define two agents now.

In [11]:
from src.multi_agent.crew import Crew

In [14]:
with Crew() as crew:
    # ---- Locations and Time Setup Agent ---- #
    LT_Agent = Agent(
        name="Locations and Time Setup Agent",
        backstory="You define locations, travel times, and guest arrival schedules.",
        task_description="Set up locations, travel times, and ensure accurate scheduling of arrivals.",
        task_expected_output="Structured location data and expected arrival times. Fourlocations:𝑉 ={𝐵,𝐺,𝑇,𝑊},where𝐵isBostonAir- port, 𝐺 is Gift shop, 𝑇 is Tailor shop, and 𝑊 is Wedding venue. 𝐵-𝐺 : 45, 𝐵-𝑇 : 30, 𝐵-𝑊 : 40, 𝐺-𝑇 : 20, 𝐺-𝑊 : 25, 𝑇-𝑊 : 15. - Alex:At𝐵at11:00AMfromChicago(needaride) - Jamie:At𝐵at12:30PMfromAtlanta(needaride) - Pat: At 𝑊 at 12:00 PM driving from NYC (has 5-seater car)"
    )
    
    # ---- Task Setup Agent ---- #
    TS_Agent = Agent(
        name="Task Setup Agent",
        backstory="You manage the scheduling of required wedding tasks.",
        task_description="Schedule gift collection after 12:00 PM, clothes pickup before 2:00 PM, and ensure photo session at 3:00 PM.",
        task_expected_output="Optimized task schedule aligned with constraints."
    )
    
    # ---- Resource Management Agent ---- #
    RM_Agent = Agent(
        name="Resource Management Agent",
        backstory="You allocate available transport resources efficiently.",
        task_description="Coordinate 5 vehicle usage and Local friend Chris(5-seater)available, for guest transportation and task fulfillment.",
        task_expected_output="Optimized 5 vehicle allocation and friend welcome ensuring timely arrivals. - Onecar(5-seater)withPat,availableafterheisBoston - LocalfriendChris(5-seater)availableafter1:30PMat𝑊"
    )
    
    # ---- Constraint Validation Agent ---- #
    CV_Agent = Agent(
        name="Constraint Validation Agent",
        backstory="You verify all scheduling constraints to ensure smooth execution.",
        task_description="Ensure all tasks are completed within operating hours and vehicle constraints are met.",
        task_expected_output="Validated schedule with no conflicts. All tasks must complete before 3:00 PM photo time - Gift store opens at 12:00 PM - Tailor closes at 2:00 PM - Two cars must accommodate all transport needs"
    )

    # ---- Supervisor Agent ---- #
    WEO_Agent = Agent(
        name="Wedding Event Oversight Agent",
        backstory="You oversee the entire wedding logistics to ensure a smooth execution of tasks.",
        task_description="Monitor and ensure all tasks are completed on time, resolving any logistical issues.",
        task_expected_output="Give a wedding scheduling plan for people, task and time."
    )

    Writer_agent = Agent(
        name="Writer Agent",
        backstory="You are a language model specialised in writing text into .json files",
        task_description="Write the json response into './p5_output.json'",
        task_expected_output="A .json file containing the given string",
        tools=write_str_to_txt,
    )
    
    # ---- Define Dependencies ---- #
    LT_Agent >> TS_Agent >> RM_Agent >> CV_Agent >> WEO_Agent >> Writer_agent

 

In [15]:
crew.run()


RUNNING AGENT: Locations and Time Setup Agent

```json
{
  "locations": {
    "B": "Boston Airport",
    "G": "Gift shop",
    "T": "Tailor shop",
    "W": "Wedding venue"
  },
  "travel_times": {
    "B-G": 45,
    "B-T": 30,
    "B-W": 40,
    "G-T": 20,
    "G-W": 25,
    "T-W": 15
  },
  "guest_arrivals": [
    {
      "name": "Alex",
      "location": "B",
      "arrival_time": "11:00 AM",
      "origin": "Chicago",
      "needs_ride": true
    },
    {
      "name": "Jamie",
      "location": "B",
      "arrival_time": "12:30 PM",
      "origin": "Atlanta",
      "needs_ride": true
    },
    {
      "name": "Pat",
      "location": "W",
      "arrival_time": "12:00 PM",
      "origin": "NYC",
      "has_car": true,
      "car_capacity": 5
    }
  ]
}
```

RUNNING AGENT: Task Setup Agent

To create an optimized task schedule aligned with the given constraints and context, we need to consider the travel times between locations and the guest arrivals. Here's the proposed schedule: